<a href="https://colab.research.google.com/github/denskoy/dolzhenkov_task/blob/main/books_authors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=59c9a3c0b3038c3fc00da218f8698484e9657c0a07b614077d76c6489eb9fcb6
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [42]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType, IntegerType
from pyspark.sql.functions import col, to_date, sum, avg, month, year, desc, count
spark = SparkSession.builder \
        .appName("book_shop") \
        .getOrCreate()

schema_book = StructType([
    StructField("book_id", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("author_id", IntegerType(), True),
    StructField("genres", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("publish_date", DateType(), True)
])
schema_author = StructType([
    StructField("author_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("birth_date", DateType(), True),
    StructField("country", StringType(), True)
])

df_books = spark.read.csv("/content/books.csv", header=True, schema=schema_book, sep=",")
df_authors = spark.read.csv("/content/authors.csv", header=True, schema=schema_author, sep=",")

#Объединение таблиц
books_authors_df = df_books.join(df_authors, on="author_id", how="inner")
books_authors_df.show()
#Топ-5 авторов, книги которых принесли наибольшую выручку
top_5_authors = books_authors_df.groupBy("author_id","name").agg(sum("price").alias("total")).orderBy(desc("total"))
#Количество книг в каждом жанре
genre_count = books_authors_df.groupBy("genres").count().orderBy(desc("count"))
#Cредняя цена книг по каждому автору
avg_book = books_authors_df.groupBy("author_id","name").avg("price").orderBy(desc("avg(price)"))
#книги, опубликованные после 2000 года
books_2000th = books_authors_df.where("publish_date >= '2000-01-01'").orderBy(desc("price"))
top_5_authors.show(5)
genre_count.show()
avg_book.show()
books_2000th.show()

+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|author_id|book_id|  title|     genres|price|publish_date|     name|birth_date|  country|
+---------+-------+-------+-----------+-----+------------+---------+----------+---------+
|        2|      1| Book_1|    Mystery|73.57|  1980-12-31| Author_2|1965-12-31|   Canada|
|        1|      2| Book_2|Non-Fiction| 41.1|  1982-12-31| Author_1|1960-12-31|    India|
|       10|      3| Book_3|    Fiction|10.63|  1984-12-31|Author_10|2005-12-31|    India|
|        9|      4| Book_4|Non-Fiction|46.31|  1986-12-31| Author_9|2000-12-31|Australia|
|        7|      5| Book_5|    Science|31.13|  1988-12-31| Author_7|1990-12-31|      USA|
|        4|      6| Book_6|Non-Fiction| 83.7|  1990-12-31| Author_4|1975-12-31|       UK|
|        6|      7| Book_7|Non-Fiction|40.36|  1992-12-31| Author_6|1985-12-31|      USA|
|        2|      8| Book_8|Non-Fiction|84.48|  1994-12-31| Author_2|1965-12-31|   Canada|
|        7